# CSE151A HW5
Zheng Zeng A14679117

Firstly, importing libraries and data 

In [1]:
import numpy as np
import math
import random
import pandas as pd
train_data = np.genfromtxt(fname="pa5train.txt", dtype=float, delimiter=' ')
test_data = np.genfromtxt(fname="pa5test.txt", dtype=float, delimiter=' ')
dictionary = np.genfromtxt(fname="pa5dictionary.txt", dtype=str)

Start to implement Boosting Algorithm

In [2]:
def weak_learner(train_set, weights):
    ans = {'idx': -1, 'err': 1, 'classifier': ''}
    for j in range(4003):
        err_plus = 0
        err_minus = 0
        for i in range(len(train_set)):
            # if word i occurs, predict 1
            if train_set[i][j] == 1:
                plus_prediction = 1
            else:
                plus_prediction = -1
            if plus_prediction != train_set[i][4003]:
                err_plus += weights[i]
            # if word i does not occur, predict 1
            if train_set[i][j] == 0:
                minus_prediction = 1
            else:
                minus_prediction = -1
            if minus_prediction != train_set[i][4003]:
                err_minus += weights[i]
        if err_plus < ans['err']:
            ans['idx'] = j
            ans['err'] = err_plus
            ans['classifier'] = 'plus'
        if err_minus < ans['err']:
            ans['idx'] = j
            ans['err'] = err_minus
            ans['classifier'] = 'minus'
    return ans

def cal_weights(train_set, weights, a_t, h_t, flag):
    new_weights = np.zeros(len(train_set))
    weights_sum = 0
    for i in range(len(train_set)):
        # firstly calculate the prediction
        if flag == 'plus':
            if train_set[i][h_t] == 1:
                prediction = 1
            else:
                prediction = -1
        else:
            if train_set[i][h_t] == 0:
                prediction = 1
            else:
                prediction = -1
        new_weights[i] = weights[i] * math.exp(-1 * a_t * train_set[i][4003] * prediction)
        weights_sum += new_weights[i]
    new_weights /= weights_sum
    return new_weights

def boosting(train_set, weights, num_round):
    d = [weights]
    h = []
    a = []
    for t in range(num_round):
        learner = weak_learner(train_set, d[t])
        h_t = {'idx':learner['idx'], 'flag':learner['classifier']}
        e_t = learner['err'] 
        a_t = 1/2*math.log((1-e_t)/e_t, math.e)
        new_weights = cal_weights(train_set, d[t], a_t, h_t['idx'], learner['classifier'])
        d.append(new_weights)
        h.append(h_t)
        a.append(a_t)
    return {'a':np.asarray(a), 'h':np.asarray(h), 'weights':np.asarray(d)}
     
def predict(data, a, h):
    sign_check = 0
    for i in range(len(h)):
        if h[i]['flag'] == 'plus':
            if data[int(h[i]['idx'])] == 1:
                prediction = 1
            else:
                prediction = -1
        else:
            if data[int(h[i]['idx'])] == 0:
                prediction = 1
            else:
                prediction = -1
        sign_check += a[i] * prediction
    if sign_check > 0:
        return 1
    else:
        return -1
    
def err(test_set, boost):
    err_cnt = 0
    for i in range(len(test_set)):
        prediction = predict(test_set[i][:4003], boost['a'], boost['h'])
        if prediction != test_set[i][4003]:
            err_cnt += 1
    return err_cnt/len(test_set)

# Problem 1

In [3]:
initial_weights = np.ones(len(train_data))
initial_weights /= len(train_data)
boost = boosting(train_data, initial_weights, 4)

Firstly, testing the correctness of the implementation

In [4]:
print(err(train_data, boost))
print(err(test_data, boost))

0.051111111111111114
0.03875968992248062


In [5]:
#df = pd.DataFrame(columns = {'t', 'training error', 'testing error'})
initial_weights = np.ones(len(train_data))
initial_weights /= len(train_data)
boost = boosting(train_data, initial_weights, 3)
#df.append({"t":3, "training error":err(train_data, boost), "testing error":err(test_data, boost)}, ignore_index=True)
print("t: %d, training error: %f, testing error %f" % (3, err(train_data, boost), err(test_data, boost)))

initial_weights = np.ones(len(train_data))
initial_weights /= len(train_data)
boost = boosting(train_data, initial_weights, 7)
#df.append({"t":7, "training error":err(train_data, boost), "testing error":err(test_data, boost)}, ignore_index=True)
print("t: %d, training error: %f, testing error %f" % (7, err(train_data, boost), err(test_data, boost)))

initial_weights = np.ones(len(train_data))
initial_weights /= len(train_data)
boost = boosting(train_data, initial_weights, 10)
#df.append({"t":10, "training error":err(train_data, boost), "testing error":err(test_data, boost)}, ignore_index=True)
print("t: %d, training error: %f, testing error %f" % (10, err(train_data, boost), err(test_data, boost)))

initial_weights = np.ones(len(train_data))
initial_weights /= len(train_data)
boost = boosting(train_data, initial_weights, 15)
#df.append({"t":15, "training error":err(train_data, boost), "testing error":err(test_data, boost)}, ignore_index=True)
print("t: %d, training error: %f, testing error %f" % (15, err(train_data, boost), err(test_data, boost)))

initial_weights = np.ones(len(train_data))
initial_weights /= len(train_data)
boost = boosting(train_data, initial_weights, 20)
#df.append({"t":20, "training error":err(train_data, boost), "testing error":err(test_data, boost)}, ignore_index=True)
print("t: %d, training error: %f, testing error %f" % (20, err(train_data, boost), err(test_data, boost)))

t: 3, training error: 0.064444, testing error 0.038760
t: 7, training error: 0.028889, testing error 0.031008
t: 10, training error: 0.015556, testing error 0.038760
t: 15, training error: 0.000000, testing error 0.023256
t: 20, training error: 0.000000, testing error 0.023256


# Problem2
Finding the first ten indices

In [8]:
keys = []
for i in range(10):
    key = boost['h'][i]['idx']
    keys.append(key)

Then, print out the words

In [10]:
for key in keys:
    print("%s (%d)"%(dictionary[key], key))

remove (3018)
language (2001)
free (1423)
university (3783)
money (2339)
linguistic (2089)
click (620)
fax (1328)
want (3899)
de (886)
